# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [14]:
# import prerequisite python modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, log_loss, roc_auc_score

In [15]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv(r"C:\Users\ibast\Downloads\adult\adult.data", header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [16]:
X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

#  shapes 
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'Y_train shape: {Y_train.shape}')
print(f'Y_test shape: {Y_test.shape}')

X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

train_test_split, the random_state ensures that the random shuffling of data during splitting happens in a consistent way every time you run the code. This parameter can be set to any integer, and as long as the value remains the same, you will get the same split every time. Useful for Reproducibility: Consistency in Results: When working with machine learning models, it’s essential to obtain consistent results. By setting a fixed random_state, we ensure that any analysis or model we develop on the split data remains reproducible by others. Comparison Across Models: If you compare multiple models or modify your approach, having the same train-test split allows you to attribute differences in results to model changes, not data variations. Ease of Collaboration: For shared projects or published work, reproducibility ensures that collaborators or reviewers can replicate your results, strengthening the credibility and reliability of findings.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

numerical_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

# Pipeline numerical 
numerical_transformer = Pipeline(steps=[
    ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),  # Impute using KNN with distance weighting
    ('robust_scaler', RobustScaler())  # Robust scaling for outlier resistance
])

# Pipeline categorical 
categorical_transformer = Pipeline(steps=[
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),  # Impute using the mode
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))  # One-hot encode, drop for binary
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [ ]:
preprocessing = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', KNNImputer(n_neighbors=7, weights="distance")),
            ('scaler', RobustScaler())
        ]), numerical_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
        ]), categorical_features)
    ]
)

# Define the pipeline
pipe = Pipeline(steps=[
    ('preprocessing', preprocessing),
    ('classifier', RandomForestClassifier())
])

# pipeline fitting
pipe.fit(X_train, Y_train)  


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [17]:
# model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # Add the ColumnTransformer for preprocessing
    ('classifier', RandomForestClassifier(random_state=42))  # Add the Random Forest classifier
])

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [18]:
from sklearn.model_selection import train_test_split

# Assuming `X` is the features DataFrame and `Y` is the target DataFrame
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

Calculate the mean of each metric. 

In [19]:
from sklearn.model_selection import cross_validate

scoring = ['neg_log_loss']

cv_results = cross_validate(model_pipeline, X_train, y_train, cv=5, scoring=scoring, return_train_score=True)

# Convert results to a DataFrame
cv_df = pd.DataFrame(cv_results)

cv_df_sorted = cv_df.sort_values(by='test_neg_log_loss', ascending=False)

# results
cv_df_sorted

c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,fit_time,score_time,test_neg_log_loss,train_neg_log_loss
3,12.708740,0.113908,-0.356791,-0.082511
0,12.483332,0.128106,-0.357675,-0.081189
1,15.979565,0.129162,-0.369239,-0.081516
2,13.308411,0.123250,-0.375988,-0.081469
4,11.884736,0.111188,-0.380379,-0.081368


In [20]:
cv_df = pd.DataFrame(cv_results)

test_metrics = cv_df.filter(regex='test_')

mean_metrics = test_metrics.mean()

print("Mean of cross-validation folds:")
print(mean_metrics)

Mean of cross-validation folds:
test_neg_log_loss   -0.368014
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [21]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

#  prediction probabilities
Y_pred_proba = pipe.predict_proba(X_test)

# Calculate binary predictions
Y_pred = pipe.predict(X_test)

# Step 3: Compute each metric
performance_metrics = {
    'negative_log_loss': -log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

print("Performance metrics on the testing data:")
print(performance_metrics)

Performance metrics on the testing data:
{'negative_log_loss': -0.37940273142901965, 'roc_auc': np.float64(0.8994978803967568), 'accuracy': 0.8540280479066434, 'balanced_accuracy': np.float64(0.7732333499701753)}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Binary Classification Compatibility: Most machine learning models, including the RandomForestClassifier, expect numerical labels for classification tasks. By converting the income column into binary values (0 and 1), where 0 represents an income of <=50K and 1 represents >50K, you make the target variable suitable for classification. Simplification: Instead of dealing with string labels like ">50K" and "<50K", the target variable is converted into numeric form directly within the data loading process. This simplifies data handling and makes it easier to apply statistical methods. Improved Model Performance: Some machine learning algorithms, such as gradient boosting and logistic regression, perform better with numerical labels for binary classification, as they can compute loss functions more efficiently. By recoding the target, these algorithms can better compute the decision boundary.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.